<a href="https://colab.research.google.com/github/DCharles01/Jigsaw-Colabs/blob/main/lesson/3-lambda-s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Triggering the Lambda Function

### Introduction

As we know, our lambda function is invoked by different events.  In this lesson, we'll set up our lambda function to be called by one of these events, which is uploading an object to a specified bucket.  

Let's get started.

> The below is adapted from the following [aws tutorial](https://docs.aws.amazon.com/lambda/latest/dg/with-s3-example.html).

### Creating a bucket

Our first step is to create an AWS bucket.  This will be the bucket that will trigger our lambda function.  

There's no special configuration here -- just set a name and use the default configurations.  

> The only thing to pay attention to is that the region you create your bucket in matches the region you'll create your lambda function in.

### Create a policy

Now that we created our triggering bucket, the next step would be to create our lambda function.  But, before we do so, let's create a custom policy to attach to our lambda function.

Go to the [policies page](https://console.aws.amazon.com/iam/home#/policies), which you can also find under the IAM resource in the web console.  And then click on create policy, and then the JSON tab.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/click-json.png?raw=1" width="100%">

Then paste in the following json:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:PutLogEvents",
                "logs:CreateLogGroup",
                "logs:CreateLogStream"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": "arn:aws:s3:::*/*"
        }
    ]
}
```

So above you can see that we added a policy with two statements.  The first is to give our lambda function access to cloudwatch, so that it can create log events when our function is called.

And the second is to allow our lambda function to read from the s3 bucket.

Ok, from there, fill in a policy name like `lambdas3trigger`.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/lambda-trigger.png?raw=1" width="100%">

And click on the blue `Create Policy` button.

Now that we have created a policy, the next step is to attach the policy to an IAM role.  So go to [roles](https://console.aws.amazon.com/iam/home#/roles), and click on `Create Role`.  

From there choose AWS service, and click the option next to `Lambda`.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/service-lambda.png?raw=1" width="100%">

From here, we can attach the custom policy that we just created.  

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/lambdas3.png?raw=1" width="100%">

From there, click `Next` and give your role a name (eg. s3-trigger-role).   

### Creating the lambda function

Ok, so we just created a role with a policy that allows our lambda function to both access S3 objects, and create new log events.  

The next step is to create the lambda function itself.  Search for lambda in the console, and click author from scratch.  Then give the function a name, and choose Python3.10 as the runtime.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/lambda-s3.png?raw=1">

You'll need to apply an existing role -- the role that we just created.  This is what will allow our lambda function to interact with S3 and cloudwatch.

### Adding the trigger

Ok, so now we can add a trigger to our lambda function.  Do so by clicking on the `Add trigger` button in the Function overview panel.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/lambda-add-trigger.png?raw=1" width="100%">

You'll see that AWS provides a variety of services that can trigger a lambda function.  However, the service we are looking for is s3.

Choose that, then specify the bucket that you want to trigger your lambda function, and under Event types choose `All object create events`.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/all-create.png?raw=1" width="80%">

> Troubleshooting: If you run into an error, it could be the configuration of your s3 bucket.

Go to the bucket, and then click on `properties`.  If you scroll down, you should see that All object create events triggers the lambda function.  However, if there were any error messages, you can try selecting and then deleting the event notification and trying to add the trigger again.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/properties.png?raw=1">

Ok, if the trigger worked, you'll see the s3 bucket added as the trigger in your lambda function's Function overview panel.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/s3-trigger.png?raw=1" width="80%">

### Testing an invocation

Ok, time to see if this thing works.  First, let's update the lambda function to the following:

```python
import json

def lambda_handler(event, context):
    print('function called')
    print('event', event)
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
```

And then click `Deploy` to deploy the changes.  From there, we can upload a file to the S3 bucket.

We can see if this worked by going to the `Monitor` panel, in our lambda function.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/cloudwatch-log.png?raw=1" width="80%">

If you click on view cloudwatch logs, and then click on one of the log streams, you should see that an event was logged.

<img src="https://github.com/data-engineering-jigsaw/aws-lambda/blob/main/lesson/cloudwatch-events.png?raw=1" width="100%">

And if you look closely at the event, you can see information about the bucket and key (ie. the object) which produced the event.

### Summary


In this lesson, we triggered calling our lambda function from uploading an s3 object.  The main step, was setting up our permissions properly -- which we did by creating a policy that had s3 read access and cloudwatch write access (to produce logs).  

Then we attached the policy to a role, and the role to our lambda function.  And from there, updated our lambda function to print out our event.  We went to cloudwatch logs to verify that when an object was uploaded to s3, it called our function.

### Resources

[AWS lambda tutorial](https://docs.aws.amazon.com/lambda/latest/dg/with-s3-example.html)

[AWS eventbridge](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-what-is.html)